In [1]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import argparse
import pdb
import HEI
import sys

In [63]:
important=['Participant ID','VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700',
       'VEG0600','VEG0900','VEG0500','VEG0100','VEG0700','FRU0100','FRU0200','FRU0300','FRU0400',
       'FRU0500','FRU0600','FRU0700','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700',
       'Whole Grains (ounce equivalents)','DMF0100','DMR0100','DML0100','DMN0100','DMF0200',
       'DMR0200','DML0200','DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
       'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100','DOT0300','DOT0400','DOT0500',
       'DOT0600','DOT0100','MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
       'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200','MFF0100','MFL0100',
       'MFF0200','MSL0100','MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400',
       'MOF0500','MOF0600','MOF0700','VEG0700','MFF0100','MFL0100','MFF0200','MSL0100','MSF0100',
       'MOF0500','MOF0600','MOF0700','VEG0700','Sodium (mg)','Refined Grains (ounce equivalents)',
       'Added Sugars (by Total Sugars) (g)','% Calories from SFA','Energy (kcal)',
       'Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
       'Total Saturated Fatty Acids (SFA) (g)','FRU0100','FRU0200','DMF0200','DMR0200','DML0200','DML0300',
       'SWT0600','BVS0400','BVS0300','BVS0500','BVS0100','BVS0200','BVS0600','BVS0700', 'SWT0600','SWT0100',
       'SWT0200','SWT0300','SWT0700' , 'SWT0800','SWT0400','SWT0500','DOT0300' , 'DOT0400',
       'DOT0100','DOT0200','GRR0800' , 'GRS0800',  'GRW0800','GRW0900' ,'GRS0900' , 'GRR0900' ,
       'GRW1100' , 'GRW1200', 'GRW0400' ,'GRS0400' ,'GRR0400','VEG0800','FMC0100', 'FMC0200','DMF0100',
       'DMR0100','DML0100','DMN0100' ,'DMF0200','DMR0200','DML0200','DOT0500', 'DOT0600','DOT0700',
       'DOT0800','GRW0600','GRS0600','GRR0600','GRW0700','GRS0700','GRR0700','GRW1300','GRS1300']

para_dict = {
        'hei_totveg': {'parameters':[1.1], 'name': 'HEIX1_TOTALVEG'},
        'hei_greensbeans': {'parameters':[0.2], 'name': 'HEIX2_GREEN_AND_BEAN'},
        'hei_totfruit': {'parameters':[0.8], 'name': 'HEIX3_TOTALFRUIT'},
        'hei_wholefruit': {'parameters':[0.4], 'name': 'HEIX4_WHOLEFRUIT'},
        'hei_wholegrains': {'parameters':[1.5], 'name': 'HEIX5_WHOLEGRAIN'},
        'hei_dairy': {'parameters':[1.3], 'name': 'HEIX6_TOTALDAIRY'},
        'hei_totproteins': {'parameters':[2.5], 'name': 'HEIX7_TOTPROT'},
        'hei_seafoodplantprot': {'parameters':[0.8], 'name': 'HEIX8_SEAPLANT_PROT'},
        'hei_refinedgrains': {'parameters':[1.8,4.3], 'name': 'HEIX11_REFINEDGRAIN'},
        'hei_addedsugars': {'parameters':[6.5,26], 'name': 'HEIX12_ADDEDSUGARS'},
        'hei_SFA': {'parameters':[8,16], 'name': 'HEIX13_SATFATS'},
        'Fats': {'parameters':[1.2,2.5], 'name': 'HEIX9_FATTYACID'},
        'hei_sodium':{'parameters':[1.1,2.0],'name':'HEIX10_SODIUM'}
        }

ped811_dict = {
    'hei_vegetables': {'parameters':[0.1,1.9], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0.1,1.9], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[1.0,3.5,0.0 , 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_milk': {'parameters':[20,28,8,35], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[2.5,6.0,0,10.0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[1.6,3.5], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[0.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
        }

ped1224_dict ={
    'hei_vegetables': {'parameters':[0.1,7.9], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0.1,7.9], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[1.5, 5.5, 0, 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_milk': {'parameters':[14.0,18.0,8.0,24.0], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[2.0,3.0,0,6.0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[1.9, 4.2], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[4.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
        }

ped08_dict = {
    'hei_vegetables': {'parameters':[0,0], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0,0], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[0,0,0,0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_dairy': {'parameters':[0,0,0,0], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[0,0,0,0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[0,0], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[0,0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0,0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0,0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0,0], 'name': 'HEIX10_SALTY'},
    'hei_cereal': {'parameters':[0,0], 'name': 'HEIX11_CEREAL'}
    }

ped_dict={
    'young': ped811_dict,
    'child':ped1224_dict,
    'infant':ped08_dict
}


hei_dict={
      'hei_totveg':
      ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
      'hei_greensbeans':
      ['VEG0100','VEG0700'],
      'hei_totfruit':
      ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
      'hei_wholefruit':
      ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
      'hei_wholegrains':
      ['Whole Grains (ounce equivalents)'],
      'hei_dairy':
      ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
                   'DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
                   'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100',
                   'DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
      'hei_totproteins':
      ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                         'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                         'MFF0100','MFL0100','MFF0200','MSL0100',
                         'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                         'MOF0600','MOF0700','VEG0700'],
      'hei_seafoodplantprot':
      ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
      'hei_sodium':
      ['Sodium (mg)'],
      'hei_refinedgrains':
      ['Refined Grains (ounce equivalents)'],
      'hei_addedsugars':
      ['Added Sugars (by Total Sugars) (g)'],
      'ripctsfa': ['% Calories from SFA','Energy (kcal)'],
     'energy':
     ['Energy (kcal)'],
     'fats':
     ['Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
     'Total Saturated Fatty Acids (SFA) (g)']
     }

hei_ped_dict={'hei_totveg':
        ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
        'hei_greensbeans':
        ['VEG0100','VEG0700'],
        'hei_totfruit':
        ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
        'hei_wholefruit':
        ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
        'hei_wholegrains':
        ['Whole Grains (ounce equivalents)'],
        'hei_dairy':
        ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
                     'DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
                     'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100',
                     'DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
        'hei_totproteins':
        ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                           'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                           'MFF0100','MFL0100','MFF0200','MSL0100',
                           'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                           'MOF0600','MOF0700','VEG0700'],
        'hei_seafoodplantprot':
        ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
        'hei_refinedgrains':
        ['Refined Grains (ounce equivalents)'],
       'energy':
       ['Energy (kcal)'],
        'hei_fruitjuice':
        ['FRU0100','FRU0200'],
        'hei_SSB':
        ['DMF0200','DMR0200','DML0200','DML0300','SWT0600','BVS0400','BVS0300','BVS0500','BVS0100','BVS0200','BVS0600','BVS0700', 'SWT0600'],
        'chocolate_candies':
        ['SWT0100'],
        'candies':
        ['SWT0200'],
        'frosting':
        ['SWT0300'],
        'sweet_sauce':
        ['SWT0700' , 'SWT0800' ],
        'sugar':
        ['SWT0400' ],
        'syrups':
        ['SWT0500' ],
        'Pudding':
        ['DOT0300' , 'DOT0400'],
        'icecream':
        ['DOT0100' ],
        'nondairy_treat':
        ['DOT0200' ],
        'baked_good':
        ['GRR0800' , 'GRS0800',  'GRW0800' ],
        'chips':
        ['GRW0900' ,'GRS0900' , 'GRR0900' , 'GRW1100' , 'GRW1200', 'GRW0400' ,'GRS0400' ,'GRR0400' ],
        'fries':
        ['VEG0800' ],
        'other_fried':
        ['FMC0100', 'FMC0200' ],
        'milk':
        ['DMF0100','DMR0100','DML0100','DMN0100' ,'DMF0200','DMR0200','DML0200','DOT0500', 'DOT0600'],
        'formula_foz':
        ['DOT0700', 'DOT0800'],
        'cereal_oz':
        ['GRW0600','GRS0600','GRR0600','GRW0700','GRS0700','GRR0700'],
        'bbcereal_hcup':
        ['GRW1300','GRS1300']
        }

conv_dict={
    'hei_fruitjuice':4,
    'hei_SSB':8,
    'chocolate_candies':40,
    'candies':15,
    'frosting':35,
    'sugar':4,
    'syrups':.25,
    'Pudding':1,
    'icecream':.5,
    'nondairy_treat':85,
    'baked_good':55,
    'fries': 70,
    'formula_foz':5,
    'bbcereal_hcup':.5
}

make_hei_dict={
    'hei_sweets' :
    ['chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good'],
    'hei_salty' :
    ['chips','other_fried','fries'],
    'hei_vegetables':
    ['hei_totveg', 'hei_greensbeans'],
    'hei_proteins':
    ['hei_totproteins','hei_seafoodplantprot'],
    'hei_milk':
    ['hei_dairy','formula_foz'],
    'hei_cereal':
    ['cereal_oz', 'bbcereal_hcup']
}

interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains','hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot', 'Total Polyunsaturated Fatty Acids (PUFA) (g)',
        'Total Monounsaturated Fatty Acids (MUFA) (g)', 'Total Saturated Fatty Acids (SFA) (g)',
        'hei_sodium', 'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa','energy','% Calories from SFA']

ped_interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains','hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot',
        'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa','energy','chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good',
        'chips','other_fried','fries']


In [3]:
def make_components(hei_dict, complete_df):
    for key, value in hei_dict.items():
        if key in ['hei_totveg','hei_greensbeans','hei_totfruit', 'hei_wholefruit']:
            #these are in cups
            x=value
            complete_df[key] = complete_df[x].astype('float').sum(axis=1)
            complete_df[key] = complete_df[key]/2
        if key in ['hei_wholegrains','hei_refinedgrains']:
            # these are in oz
            x=value
            complete_df[key] = complete_df[x].astype('float')
        if key in ['hei_dairy']:
            # these are in cups
            x=value[:-1]
            tmp= complete_df[x].astype('float').sum(axis=1)
            y=value[-1]
            if y == 'DOT0100':
                tmp2=complete_df[y].astype('float')/3
                complete_df[key]=tmp+tmp2
            else:
                print('NO DAIRY MISSING DOT0100, needs to be last in list')
        if key in ['hei_totproteins']:
            # these are in oz
            x=value[:-1]
            tmp= complete_df[x].astype('float').sum(axis=1)
            y=value[-1]
            if y == 'VEG0700':
                tmp2=complete_df[y].astype('float')*2 # this is normally 1/2
                complete_df[key]=tmp+tmp2
            else:
                print('NO TOTAL PROTEIN MISSING VEG0700, needs to be last in list')
        if key in ['hei_seafoodplantprot']:
            # these are in oz
            x=value[:-1]
            tmp= complete_df[x].astype('float').sum(axis=1)
            y=value[-1]
            if y == 'VEG0700':
                tmp2=complete_df[y].astype('float')*2
                complete_df[key]=tmp+tmp2
            else:
                print('NO SEAFOOD AND PLANT PROTEIN MISSING VEG0700, needs to be last in list')
        if key in ['hei_sodium']:
            x=value
            complete_df[key] = complete_df[x].astype('float')/1000
        if key in ['hei_addedsugars']:
            x=value
            complete_df[key] = complete_df[x].astype('float')*4
        if key in ['ripctsfa']:
            x=value[0]
            y=value[1]
            complete_df[key] = complete_df[x].astype('float')*complete_df[y].astype('float')
        if key in ['energy']:
            x=value
            complete_df[key] = complete_df[x].astype('float')/1000
    return(complete_df)

In [4]:
basepath='/Users/gracer/Google Drive/BCP/data'

In [64]:
arglist={'BASEPATH': basepath, 
         'CHILD': False, 
         'NAMES': 'testies', 
         'OPTS': ['Mom'], 
         'SAVE':os.path.join(basepath, 'derivived'),
         'XTRA': False}

In [65]:
for (dirpath, dirnames, filenames) in os.walk(basepath):
        for filename in filenames:
            if filename.endswith('.zip'):
                tmppath=os.sep.join([dirpath, filename])
                with ZipFile(tmppath, 'r') as zipObj:
                   # Get a list of all archived file names from the zip
                   listOfFileNames = zipObj.namelist()
                   # Iterate over the file names
                   for fileName in listOfFileNames:
                       # Check filename endswith txt
                        if fileName.endswith('04.txt'):
                            zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
                        if fileName.endswith('09.txt'):
                            zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
infile = os.path.join(basepath,'temp_txt')

In [78]:
x=HEI.file_org(infile, arglist, important)

this is the item Mom
Mom
Mom


In [79]:
x

{'set_04': {'Mom': {'files': ['/Users/gracer/Google Drive/BCP/data/temp_txt/BCPMomMSTR04.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNMomMar1804.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/MomOct1804.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/BCPJulAugMom04.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNMomJuly04.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UNCMomJuly04.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCMom04.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/BCPMomAP04.txt']}},
 'set_09': {'Mom': {'files': ['/Users/gracer/Google Drive/BCP/data/temp_txt/BCPMomAP09.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCMom09.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UNCMomJuly09.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNMomJuly09.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/BCPJulAugMom09.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/M

In [99]:
data_dict={"set_04":{},"set_09":{}}

for key, value in x.items():
    print(key)
    for k,v, in value.items():
        print(k)
        temp_list = []
        for file in v["files"]:
            print(file)
            temp_df =  pd.read_csv(file, sep="\t", encoding='latin1')
            temp_df=temp_df.drop([0])
            for val in temp_df["Participant ID"]:
                _id = str(val).lstrip("0").split("_")[0]
                temp_df.replace(val, _id, inplace=True)
            temp_list.append(temp_df)
#             pdb.set_trace()
        dfm_original = pd.concat(temp_list, ignore_index=True)
        print("Final dataframe size: ", dfm_original.shape)
        dfm_original = dfm_original.sort_values(by="Participant ID")

        for col in dfm_original:
            if dfm_original[col].dtype == np.object_:
                dfm_original[col] = (dfm_original[col].replace(',','.', regex=True))   
        
        concat_filepath = os.path.join(arglist['SAVE'],'%s_dataset_%s.csv'%(k,key))
        dfm_original.to_csv(concat_filepath, index=False, sep=",", header=True)
        
        data_dict[key]= dfm_original


set_04
Mom
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPMomMSTR04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNMomMar1804.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/MomOct1804.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPJulAugMom04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNMomJuly04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UNCMomJuly04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCMom04.txt
/Users/gracer/Google Drive/BCP/data/temp_txt/BCPMomAP04.txt
Final dataframe size:  (360, 226)
Project Abbreviation        object
Participant ID              object
Date of Intake              object
Project Name               float64
Record Type                  int64
                            ...   
Data Field 3 Response      float64
Data Field 4 Descriptor    float64
Data Field 4 Response      float64
Data Field 5 Descriptor    float64
Data Field 5 Response      float64
Length: 226, dtype: object
set_09
Mom
/Users/gracer/Google Drive/BCP/data/te

In [100]:
data_dict['set_09'].iloc[:,3:]

,FRU0100,FRU0200,FRU0300,FRU0400,FRU0500,FRU0600,FRU0700,VEG0100,VEG0200,VEG0300,...,MSC0300,MSC0400,MSC0500,MSC0600,MSC0700,MSC0800,MSC0900,MSC1000,GRW1300,GRS1300
308,0,0,0,3,0,0,0,2,2.295,1.68,...,0,0,0,0,0,0,0,0,0,0
48,0,0,0,3,0,0,0,2,2.295,1.68,...,0,0,0,0,0,0,0,0,0,0
309,0.012,0,0,2.146,0.576,0,0,0,0,0.383,...,0,0.425,0.659,0,0,0,0,0,0,0
310,0.29,0,0,2.338,0,0,0,0.89,0,1.377,...,0,0,0,0,0,0,0,0,0,0
311,0.023,0,0,1.343,0.265,0,0,0.554,0,0.419,...,0,3,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2,0,0,4.758,0,0,0,3,1.865,0,...,0,0,0,0,0,0,0,0,0,0
191,0,0,0,1.25,0,0,0,3,0,1.408,...,0,0,0,0,0,0,0,0,0,0
69,0,0,0,0,0,0,0,0,0.009,0.347,...,0,0,0,0,0,0,0,0,0,0
226,0,0,0.966,0,1.813,0,0,0,0,1.367,...,0,0,0,0,0,0.249,0,0,0,0


In [101]:
def make_components(hei_dict, data_dict):
    for sets, _df in data_dict.items():
        if sets == 'set_04':
            for key, value in hei_dict.items():
                if key in ['hei_wholegrains','hei_refinedgrains']:
                    # these are in oz
                    x=value
                    _df[key] = _df[x].astype('float')
                if key in ['hei_sodium']:
                    x=value
                    _df[key] = _df[x].astype('float')/1000
                if key in ['hei_addedsugars']:
                    x=value
                    _df[key] = _df[x].astype('float')*4
                if key in ['ripctsfa']:
                    x=value[0]
                    y=value[1]
                    _df[key] = _df[x].astype('float')*_df[y].astype('float')
                if key in ['energy']:
                    x=value
                    _df[key] = _df[x].astype('float')/1000
                    
        if sets == 'set_09':
            print('start set 09')
            for key, value in hei_dict.items():
                if key in ['hei_totveg','hei_greensbeans','hei_totfruit', 'hei_wholefruit']:
                    #these are in cups
                    x=value
                    _df[key] = _df[x].astype('float').sum(axis=1)
                    _df[key] = _df[key]/2
                if key in ['hei_dairy']:
                    # these are in cups
                    x=value
                    tmp= _df[x].astype('float').sum(axis=1)
                    y=value[-1]
                    if y == 'DOT0100':
                        tmp2= _df[y].astype('float')/3
                        _df[key]=tmp+tmp2
                    else:
                        print('NO DAIRY MISSING DOT0100, needs to be last in list')
                if key in ['hei_totproteins']:
                    # these are in oz
                    x=value
                    tmp = _df[x].astype('float').sum(axis=1)
                    y=value[-1]
                    if y == 'VEG0700':
                        tmp2 = _df[y].astype('float')*2 # this is normally 1/2
                        _df[key]=tmp+tmp2
                    else:
                        print('NO TOTAL PROTEIN MISSING VEG0700, needs to be last in list')
                if key in ['hei_seafoodplantprot']:
                    # these are in oz
                    x=value
                    tmp= _df[x].astype('float').sum(axis=1)
                    y=value[-1]
                    if y == 'VEG0700':
                        tmp2 = _df[y].astype('float')*2
                        _df[key]=tmp+tmp2
                    else:
                        print('NO SEAFOOD AND PLANT PROTEIN MISSING VEG0700, needs to be last in list')
                
    return(data_dict)

In [102]:
df_dict=make_components(hei_dict, data_dict)

start set 09


In [106]:
df_dict['set_04'].describe()

,Project Name,Record Type,Participant Name,Gender,Date of Birth,Date of Entry,Day of Intake,Visit Number,Interviewer ID,Site ID,...,Data Field 4 Descriptor,Data Field 4 Response,Data Field 5 Descriptor,Data Field 5 Response,hei_wholegrains,hei_sodium,hei_refinedgrains,hei_addedsugars,ripctsfa,energy
count,0.0,360.0,0.0,360.0,0.0,0.0,360.000000,359.000000,0.0,1.0,...,0.0,0.0,0.0,0.0,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000
mean,NaN,1.0,NaN,3.0,NaN,NaN,2.505556,1.757660,NaN,1.0,...,NaN,NaN,NaN,NaN,2.295517,3.677125,5.408653,225.386333,28484.462374,2.286764
std,NaN,0.0,NaN,0.0,NaN,NaN,1.630138,1.024717,NaN,NaN,...,NaN,NaN,NaN,NaN,2.223464,1.331191,3.324019,147.684359,13780.279707,0.722255
min,NaN,1.0,NaN,3.0,NaN,NaN,0.000000,1.000000,NaN,1.0,...,NaN,NaN,NaN,NaN,0.000000,0.928639,0.000000,1.388000,1877.543520,0.431520
25%,NaN,1.0,NaN,3.0,NaN,NaN,1.000000,1.000000,NaN,1.0,...,NaN,NaN,NaN,NaN,0.525500,2.812654,2.924000,119.117000,18465.564658,1.723783
50%,NaN,1.0,NaN,3.0,NaN,NaN,3.000000,1.000000,NaN,1.0,...,NaN,NaN,NaN,NaN,1.850000,3.525236,4.958000,199.926000,26297.009936,2.237577
75%,NaN,1.0,NaN,3.0,NaN,NaN,4.000000,2.000000,NaN,1.0,...,NaN,NaN,NaN,NaN,3.581000,4.380383,6.970750,299.616000,34263.494079,2.772505
max,NaN,1.0,NaN,3.0,NaN,NaN,6.000000,6.000000,NaN,1.0,...,NaN,NaN,NaN,NaN,12.127000,10.302905,16.889000,799.284000,75753.445320,5.342274


In [105]:
df_dict['set_09'].describe()

,hei_totveg,hei_greensbeans,hei_totfruit,hei_wholefruit,hei_dairy,hei_totproteins,hei_seafoodplantprot
count,368.000000,368.000000,368.000000,368.000000,368.000000,368.000000,368.000000
mean,1.989827,0.524370,1.031329,0.885957,2.359358,7.287736,2.532418
std,1.559309,0.813705,1.044187,0.940544,1.877176,4.330759,3.045076
min,0.000000,0.000000,0.000000,0.000000,0.000000,-0.006000,-0.847000
25%,0.934500,0.000000,0.093000,0.000000,0.980000,3.985000,0.000000
50%,1.627000,0.208750,0.718250,0.558250,1.838000,6.856000,1.682500
75%,2.863000,0.750000,1.548000,1.378625,3.378000,10.056250,3.875000
max,11.517000,6.316500,4.950000,4.942000,14.396000,30.404000,18.001000


In [107]:
def make_hei( make_hei_dict):
    print('START')
    print(make_hei_dict)
    for k, value in make_hei_dict.items():
        print(k)
        x=value
#         complete_df[k] = complete_df[x].astype('float').sum(axis=1)
    return()

In [110]:
def check(dic, data, name, option, arglist):
    df=data
    toSum=['HEIX1_TOTALVEG','HEIX2_GREEN_AND_BEAN' , 'HEIX3_TOTALFRUIT' , 'HEIX4_WHOLEFRUIT' ,
           'HEIX5_WHOLEGRAIN' , 'HEIX6_TOTALDAIRY' , 'HEIX7_TOTPROT' , 'HEIX8_SEAPLANT_PROT' , 'HEIX9_FATTYACID' ,
           'HEIX10_SODIUM' , 'HEIX11_REFINEDGRAIN' , 'HEIX12_ADDEDSUGARS' , 'HEIX13_SATFATS']
    for key,values in dic.items():
        print('Calculating score for %s'%key)
        if key in ['hei_totveg','hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_totproteins', 'hei_seafoodplantprot',
                    'hei_wholegrains', 'hei_dairy','Fats']:
            if key == 'Fats':
                adeq_check(df, key, values['name'], values['parameters'])
            else:
                adeq_check(df, key, values['name'], values['parameters'][0])
        if key in ['hei_sodium','hei_refinedgrains','hei_addedsugars','hei_SFA']:
            mod_check(df, key, values['name'], values['parameters'])
    df['HEI2015_TOTAL_SCORE']=df[df.columns.intersection(toSum)].sum(axis=1)
    concat_filepath = os.path.join(arglist['SAVE'],'%s_%s_HEI.csv'%(option, name))
    df.to_csv(concat_filepath, index=False, sep=",", header=True)


In [114]:
for k, item in df_dict.items():
    print(k)
    check(para_dict, item, k, key, arglist)

set_04
Calculating score for hei_totveg


KeyError: 'hei_totveg'

In [112]:
def adeq_check(df,inputt, output, parameter):
    # fruit, vegetables, greens and beans, dairy are in cup/1000cal
    # grains, protein are in oz
    if inputt in ['hei_totveg','hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_totproteins', 'hei_seafoodplantprot']:
        tmp = df[inputt]/df['energy']
        df[output] = [5 if x >= parameter else 5*(x/parameter) for x in tmp]
    elif inputt in ['hei_wholegrains', 'hei_dairy']:
        if inputt == 'hei_wholegrains':
            tmp = df[inputt]/df['energy']
            df[output] = [10 if x >= parameter else 10*(x/parameter) for x in tmp]
        else:
            tmp = df[inputt]/df['energy']
            df[output] = [10 if x >= parameter else 10*(x/parameter) for x in tmp]
    elif inputt in ['Fats']:
        FARMIN=parameter[0]
        FARMAX=parameter[1]
        tmp=df['Total Polyunsaturated Fatty Acids (PUFA) (g)']+df['Total Monounsaturated Fatty Acids (MUFA) (g)']
        tmp2=tmp/df['Total Saturated Fatty Acids (SFA) (g)']
        df[output] = [10 if x > FARMAX else 0 if x <= FARMIN else 10*((x-FARMIN)/(FARMAX-FARMIN)) for x in tmp2]


In [11]:
for key,value in x.items():
    print(value)
    y=HEI.make_components(hei_dict, value)
    test=['hei_totveg','hei_greensbeans','hei_totfruit','hei_wholefruit','hei_wholegrains','hei_dairy','hei_totproteins','hei_seafoodplantprot','hei_refinedgrains','hei_addedsugars','hei_sodium']
    y=y[(y[test] > 0).all(1)]
    z=HEI.grouper(y, interest, arglist)

{'Mom': {'files': ['/Users/gracer/Google Drive/BCP/data/temp_txt/BCPMomMSTR04.txt', '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNMomMar1804.txt', '/Users/gracer/Google Drive/BCP/data/temp_txt/MomOct1804.txt', '/Users/gracer/Google Drive/BCP/data/temp_txt/BCPJulAugMom04.txt', '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNMomJuly04.txt', '/Users/gracer/Google Drive/BCP/data/temp_txt/UNCMomJuly04.txt', '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCMom04.txt', '/Users/gracer/Google Drive/BCP/data/temp_txt/BCPMomAP04.txt']}}


TypeError: unhashable type: 'list'

In [ ]:
for k, item in z.items():
    HEI.check(para_dict, item, k, key, arglist)

In [8]:
y=HEI.make_components(hei_dict, x['Mom'])

In [9]:
y.columns

Index(['Project Abbreviation', 'Participant ID', 'Date of Intake',
       'Project Name', 'Record Type', 'Gender', 'Date of Entry',
       'Day of Intake', 'Visit Number', 'Interviewer ID',
       ...
       'hei_wholefruit', 'hei_wholegrains', 'hei_dairy', 'hei_totproteins',
       'hei_seafoodplantprot', 'hei_sodium', 'hei_refinedgrains',
       'hei_addedsugars', 'ripctsfa', 'energy'],
      dtype='object', length=395)

In [16]:
test=['hei_totveg','hei_greensbeans','hei_totfruit','hei_wholefruit','hei_wholegrains','hei_dairy','hei_totproteins','hei_seafoodplantprot','hei_refinedgrains','hei_addedsugars','hei_sodium']
test4=y[(y[test] > 0).all(1)]






In [18]:
test5=test4[['hei_totveg','hei_greensbeans','hei_totfruit','hei_wholefruit','hei_wholegrains','hei_dairy','hei_totproteins','hei_seafoodplantprot','hei_refinedgrains','hei_addedsugars','hei_sodium']]
test5.describe()

,hei_totveg,hei_greensbeans,hei_totfruit,hei_wholefruit,hei_wholegrains,hei_dairy,hei_totproteins,hei_seafoodplantprot,hei_refinedgrains,hei_addedsugars,hei_sodium
count,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.00000,119.000000,119.000000,119.000000
mean,2.437933,0.974660,1.364878,1.175235,2.939748,2.514947,8.770908,4.32900,4.834403,220.622319,3.785809
std,1.775489,1.024844,1.051552,0.926007,2.354186,1.508472,4.154725,3.44538,3.211498,127.979574,1.471337
min,0.280000,0.009000,0.045500,0.044000,0.181000,0.235000,1.091000,0.04300,0.125000,1.388000,0.928639
25%,1.130750,0.399250,0.558250,0.500000,1.279000,1.421500,5.745500,1.98600,2.525500,119.848000,2.662591
50%,2.089500,0.608000,1.108000,0.961000,2.263000,2.000000,8.367000,3.00000,4.652000,197.992000,3.760789
75%,3.454500,1.101250,1.841000,1.558750,4.240000,3.402500,11.609500,5.92550,6.113000,309.150000,4.674946
max,11.517000,6.316500,4.950000,4.942000,12.127000,7.346000,21.499000,18.00100,16.348000,586.268000,10.302905


In [10]:
test=y[['hei_totveg','hei_greensbeans','hei_totfruit','hei_wholefruit','hei_wholegrains','hei_dairy','hei_totproteins','hei_seafoodplantprot','hei_refinedgrains','hei_addedsugars','hei_sodium']]


test.describe()


,hei_totveg,hei_greensbeans,hei_totfruit,hei_wholefruit,hei_wholegrains,hei_dairy,hei_totproteins,hei_seafoodplantprot,hei_refinedgrains,hei_addedsugars,hei_sodium
count,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000
mean,2.014612,0.547172,1.003851,0.857146,2.407559,2.412179,7.426950,2.548637,5.336789,225.862807,3.732383
std,1.616535,0.834534,1.049677,0.936472,2.278492,1.901189,4.473283,3.098313,3.349554,151.219934,1.366889
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.006000,-0.847000,0.000000,1.388000,0.928639
25%,0.804875,0.000000,0.072875,0.000000,0.581000,1.053417,3.973000,0.000000,2.715750,117.332000,2.851483
50%,1.642500,0.249000,0.687000,0.514500,1.908000,1.885500,7.032000,1.682500,4.916000,199.926000,3.568920
75%,2.872000,0.750000,1.500000,1.250000,3.659250,3.437750,10.334750,3.985000,6.949000,301.702000,4.494743
max,11.517000,6.316500,4.950000,4.942000,12.127000,14.396000,30.404000,18.001000,16.889000,799.284000,10.302905


In [14]:
test2=y[['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                             'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                             'MFF0100','MFL0100','MFF0200','MSL0100',
                             'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                             'MOF0600','MOF0700','VEG0700']]
test2=test2.astype(np.float32)
test2.describe()

,MRF0100,MRL0100,MRF0200,MRL0200,MRF0300,MRL0300,MRF0400,MRL0400,MCF0200,MCL0200,...,MCF0100,MCL0100,MOF0100,MOF0200,MOF0300,MOF0400,MOF0500,MOF0600,MOF0700,VEG0700
count,322.000000,322.000000,322.0,322.0,322.000000,322.0,322.000000,322.000000,322.000000,322.000000,...,322.000000,322.000000,322.0,322.0,322.000000,322.0,322.000000,322.000000,322.000000,322.000000
mean,0.811699,0.157736,0.0,0.0,0.002127,0.0,0.285047,0.022137,0.065807,0.055370,...,0.285531,0.333984,0.0,0.0,0.803059,0.0,0.814370,0.460103,0.100845,0.236323
std,1.469377,0.774764,0.0,0.0,0.038174,0.0,1.013837,0.244023,0.223548,0.311397,...,0.795088,0.951200,0.0,0.0,1.770700,0.0,1.857649,0.957514,0.566785,0.471878
min,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,-0.847000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
75%,1.121750,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,1.500000,0.0,0.753750,0.500000,0.000000,0.263000
max,8.000000,4.268000,0.0,0.0,0.685000,0.0,6.402000,4.000000,1.693000,3.000000,...,6.000000,4.741000,0.0,0.0,26.000000,0.0,15.619000,6.000000,4.938000,2.551000


In [15]:
test3=y[
          ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700']]
test3=test3.astype(np.float32)
test3.describe()

,MFF0100,MFL0100,MFF0200,MSL0100,MSF0100,MOF0500,MOF0600,MOF0700,VEG0700
count,322.000000,322.000000,322.000000,322.000000,322.0,322.000000,322.000000,322.000000,322.000000
mean,0.022335,0.367516,0.011174,0.063326,0.0,0.814370,0.460103,0.100845,0.236323
std,0.244951,1.206664,0.200509,0.356389,0.0,1.857649,0.957514,0.566785,0.471878
min,0.000000,0.000000,0.000000,0.000000,0.0,-0.847000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.0,0.753750,0.500000,0.000000,0.263000
max,3.000000,7.727000,3.598000,3.624000,0.0,15.619000,6.000000,4.938000,2.551000


In [ ]:
x=HEI.file_org(infile, arglist, important)
for key,value in x.items():
    y=HEI.make_components(hei_dict, value)
    z=HEI.grouper(y, interest, arglist)
    for k, item in z.items():
        HEI.check(para_dict, item, k, key, arglist)